# Projeto 1 - Ciência dos Dados

Nome: Athur Fonseca

Nome: Caio Tieri

Nome: Leonardo Andrade

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
#import emoji

#.get_emoji_regexp

In [ ]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [ ]:
filename = 'Bis 1-340.xlsx'

In [ ]:
train = pd.read_excel(filename)
train.head(5)


In [ ]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

___
## Classificador automático de sentimento


# Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.



O produto é um chocolate marcado por trazer um sentimento de insaciedade e vontade de comer sempre mais um.

# Para julgar a relevância dos tweets seguimos o critério abaixo:

1(Irrelevante): Não cita o produto ou cita o nome 'bis' mas se referindo a outra coisa.

2(neutro): Fala do produto mas este não se encaixa como o assunto principal do tweet

2(relevante): Cita o produto como assunto principal, expressa indiretamente uma opinião sobre o produto

4(muito relevante): Cita o produto  como assunto principal do tweet e expressa uma opinião clara sobre o mesmo.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [ ]:
treino = train.iloc[:,[0,1]]
treino.tail(20)

In [ ]:
#importando funções

import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;|"]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed
    
def tiraarroba (string):
    palavras = []
    lista = string.split()
    for p in lista:
        if p[0] == '@':
            palavras.append(p)
    for palavra in palavras:
        lista.remove(palavra)
    limpo = ' '.join(lista)
    return limpo

def tiralink (string):
    palavras = []
    lista = string.split()
    for p in lista:
        if p[0] == 'h' and p[1] == 't' and p[2] == 't' and p[3] == 'p' and p[4] == 's':
            palavras.append(p)
    for palavra in palavras:
        lista.remove(palavra)
    limpo = ' '.join(lista)
    return limpo

def tiralinha (string):
    palavras = []
    lista = string.split()
    for p in lista:
        limpo = p.rstrip("\n")
        palavras.append(limpo)
    tweet = " ".join(palavras)
    return tweet


print(tiralinha('eu quero replay kkkk \nvamo de novo no bis'))



# Removendo caracteres indesejados

In [ ]:
treino['Treinamento'] = treino['Treinamento'].apply(cleanup)

In [ ]:
#tira os @fulano citados

i=0

for t in treino.Treinamento:
   limpo = tiraarroba(t)
   treino.loc[i,'Treinamento'] = limpo
   i+=1
    
    


In [ ]:
#retira os links (https)

i=0

for t in treino.Treinamento:
   limpo = tiralink(t)
   treino.loc[i,'Treinamento'] = limpo
   i+=1
    

In [ ]:
treino.head(50)

In [ ]:
irrelevante = treino.loc[treino['Nível de Relevância']==1,['Treinamento']]

neutro = treino.loc[treino['Nível de Relevância']==2,['Treinamento']]

relevante = treino.loc[treino['Nível de Relevância']==3,['Treinamento']]

muito_relevante = treino.loc[treino['Nível de Relevância']==4,['Treinamento']]

In [ ]:
lista_palavras=[]

for tweet in treino['Treinamento']:
    lista = tweet.split()
    for p in lista:
        lista_palavras.append(p)

lista_irrelevante=[]

for tweet in irrelevante['Treinamento']:
    lista2 = tweet.split()
    for p in lista2:
        lista_irrelevante.append(p)

lista_neutro = []

for tweet in neutro['Treinamento']:
    lista3 = tweet.split()
    for p in lista3:
        lista_neutro.append(p)

lista_relevante = []

for tweet in relevante['Treinamento']:
    lista4 = tweet.split()
    for p in lista4:
        lista_relevante.append(p)

lista_muito_relevante = []

for tweet in muito_relevante['Treinamento']:
    lista5 = tweet.split()
    for p in lista5:
        lista_muito_relevante.append(p)

In [ ]:
serie_irrelevante = pd.Series(lista_irrelevante)
tabela_irrelevante = serie_irrelevante.value_counts(True)

serie_neutro = pd.Series(lista_neutro)
tabela_neutro = serie_neutro.value_counts(True)

serie_relevante = pd.Series(lista_relevante)
tabela_relevante = serie_relevante.value_counts(True)

serie_muito_relevante = pd.Series(lista_muito_relevante)
tabela_muito_relevante = serie_muito_relevante.value_counts(True)

In [ ]:
P_irrelevante = len(lista_irrelevante)/len(lista_palavras)

P_neutro = len(lista_neutro)/len(lista_palavras)

P_relevante = len(lista_relevante)/len(lista_palavras)

P_muito_relevante = len(lista_muito_relevante)/len(lista_palavras)

print(P_irrelevante)

print(P_neutro)

print(P_relevante)

print(P_muito_relevante)

print(P_irrelevante + P_neutro + P_relevante + P_muito_relevante)

In [ ]:
lista_todas_tabelas = [tabela_irrelevante, tabela_neutro, tabela_relevante, tabela_muito_relevante]

lista_todas_listas = [lista_irrelevante, lista_neutro, lista_relevante, lista_muito_relevante]

In [ ]:
tabela_irrelevante.to_frame().head(20)

In [ ]:
def classificador(tweet, tabelas, listas):
    p = 1
    i = 0
    soma = 0
    z = 0
    lista_resultado = []
    vezes = 1
    for tabela in tabelas:
        soma = 0
        z += 1
        vezes = 1
        for lista in listas:
            for palavra in tweet.split():
                if soma >= len(tweet.split()) :
                    break
                if palavra in tabela:
                    p = (1 + tabela[palavra])
                    soma += 1
                else:
                    p = 1
                    soma += 1
                i += 1
                vezes *= p
        lista_resultado.append(vezes)
    if lista_resultado.index(max(lista_resultado)) == 0:
        return 'irrelevante'
    elif lista_resultado.index(max(lista_resultado)) == 1:
        return 'neutro'
    elif lista_resultado.index(max(lista_resultado)) == 2:
        return 'relevante'
    elif lista_resultado.index(max(lista_resultado)) == 3:
        return 'muito relevante'

print(classificador('coco coco coco coco bis', lista_todas_tabelas, lista_todas_listas))

In [ ]:
lista_treino_classificada = []
i = 0
for a in treino['Treinamento']:
    lista_treino_classificada.append(classificador(a, lista_todas_tabelas, lista_todas_listas))
treino["classificador"] = lista_treino_classificada
treino.head(40)

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [205]:
test.head()
teste = test.iloc[:,[0,1]]
teste['Teste']= teste['Teste'].apply(cleanup)

#tira os @fulano citados

i=0

for t in teste.Teste:
   limpo = tiraarroba(t)
   teste.loc[i,'Teste'] = limpo
   i+=1

#retira os links (https)

i=0

for t in teste.Teste:
   limpo = tiralink(t)
   teste.loc[i,'Teste'] = limpo
   i+=1

teste['Teste'] = teste.Teste.apply(tiralinha)

teste.head()

    


<ipython-input-205-59578dc44b4f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  teste['Teste']= teste['Teste'].apply(cleanup)
C:\Users\caiog\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-205-59578dc44b4f>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

,Teste,Nível de Relevância
0,eu quero replay kkkk vamo de novo no bis 🤣🤣🤣🖤 ...,1
1,juro acharam uma garrafa de cachaça e beberam ...,2
2,sabia que você ia responder bis fã,1
3,olá queremos muito te ajudar para que possamos...,1
4,mudaram as fraldas a esse monte de esterco 🤢,1


In [207]:
lista_teste_classificada = []
i = 0
for tweet in teste["Teste"]:
    lista_teste_classificada.append(classificador(tweet, lista_todas_tabelas, lista_todas_listas))

teste['classificado'] = lista_teste_classificada



<ipython-input-207-482c4f8b1581>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  teste['classificado'] = lista_teste_classificada


In [209]:
def converteemnumero(nivel):
    if nivel == 'irrelevante':
        return 1
    elif nivel == 'neutro':
        return 2
    elif nivel == 'relevante':
        return 3
    else:
        return 4


valorestestados = []
for x in teste['Nível de Relevância']:
    valorestestados.append(x)

valorescalculados = []
for y in teste.classificado:
    valor = converteemnumero(y)
    valorescalculados.append(valor)

acertos = 0
total = 0
i=0
while i <= len(valorescalculados)-1:
    if valorestestados[i] == valorescalculados[i]:
        acertos+=1
    total+=1
    i+=1

qualidade = (acertos/total)*100

print(" acertou em {}% das vezes".format(qualidade))

 acertou em 10.285714285714285% das vezes


___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**